# Neural Machine Translation
Automatic data manipulation using NLP.  Convert various date structures to consistent machine-readable format. 
Use Keras, RNN, LSTM, Bidirectional, 

In [1]:
### Import modules
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
### Set up fake module instance
fake = Faker()
fake.seed(12345)
random.seed(12345)

# Define format of the data we would like to generate
FORMATS = ['short','medium','long','full','d MMM YYY', 'd MMMM YYY','dd MMM YYY','d MMM, YYY',
           'd MMMM, YYY','dd, MMM YYY','d MM YY','d MMMM YYY','MMMM d YYY','MMMM d, YYY','dd.MM.YY']

In [3]:
### Select 10,000 samples
m = 10000

### Set up outputs   
human_vocab2 = set()
machine_vocab2 = set()
dataset = []
Tx = 30

for i in tqdm(range(m)):

    dt = fake.date_object()

    human_readable1 = format_date(dt, format=random.choice(FORMATS),  locale='en_US') 
    human_readable2 = human_readable1.lower()
    human_readable = human_readable2.replace(',','')
    machine_readable = dt.isoformat()
    
    if human_readable is not None:
        dataset.append((human_readable, machine_readable))
        human_vocab2.update(tuple(human_readable))
        machine_vocab2.update(tuple(machine_readable))
    
human_vocab = dict(zip(sorted(human_vocab2) + ['<unk>', '<pad>'], 
                     list(range(len(human_vocab2) + 2))))
inv_machine_vocab = dict(enumerate(sorted(machine_vocab2)))
machine_vocab = {v:k for k,v in inv_machine_vocab.items()}


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 9162.04it/s]


In [4]:
dataset[:10]

[('09 may 1998', '1998-05-09'),
 ('10 september 1970', '1970-09-10'),
 ('4/28/90', '1990-04-28'),
 ('january 26 1995', '1995-01-26'),
 ('march 7 1983', '1983-03-07'),
 ('may 22 1988', '1988-05-22'),
 ('8 jul 2008', '2008-07-08'),
 ('september 8 1999', '1999-09-08'),
 ('1 january 1981', '1981-01-01'),
 ('22.05.95', '1995-05-22')]

In [5]:
### convert string to numeric representation
def string_to_int(string, length, vocab):

    #make lower to standardize
    string = string.lower()
    string = string.replace(',','')
    
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    return rep


In [36]:
### Set up X and Y input length
Tx = 30
Ty = 10

# separate X and Y
X1, Y1 = zip(*dataset)
    
# Prepare input variables
X2 = np.array([string_to_int(i, Tx, human_vocab) for i in X1])
Y2 = [string_to_int(t, Ty, machine_vocab) for t in Y1]
    
Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X2)))
Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y2)))

Y = np.array(Y2)

print("X.shape:", X2.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


In [7]:
### Define softmax function
def softmax(x, axis=1):
 
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s

In [8]:
### Define size of a and s states
n_a = 32
n_s = 64

### Build Keras Recurrent Neural Network Model

human_vocab_size = len(human_vocab)
machine_vocab_size = len(machine_vocab)

# Define the inputs of your model with a shape (Tx,)
# Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
X = Input(shape=(Tx, human_vocab_size), name= 'X')
s0 = Input(shape=(n_s,), name='s0')
c0 = Input(shape=(n_s,), name='c0')
s = s0
c = c0
    
# Initialize empty list of outputs
outputs = []

# Define your pre-attention Bi-LSTM. Remember to use return_sequences=True.
a = Bidirectional(LSTM(n_a, return_sequences=True, name='bidirectional_1'), merge_mode='concat')(X)
    
# Iterate for Ty steps
for t in range(Ty):
    
    # Perform one step of the attention mechanism to get back the context vector at step t
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" 
    s_prev1 = RepeatVector(Tx)(s)
   
    # Use concatenator to concatenate a and s_prev on the last axis 
    concat = Concatenate(axis=-1)([a, s_prev1])
    
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e   
    e = Dense(10, activation = "tanh")(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. 
    energies = Dense(1, activation = "relu")(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" 
    alphas = Activation(softmax)(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell 
    context = Dot(axes = 1)([alphas, a])
        
    # Apply the post-attention LSTM cell to the "context" vector.
    s, _, c = LSTM(n_s, return_state = True)(context, initial_state = [s, c])
        
    # Apply Dense layer to the hidden state output of the post-attention LSTM
    out = Dense(len(machine_vocab), activation=softmax)(s)
    
    # Append "out" to the "outputs" list
    outputs.append(out)
    
# Create model instance taking three inputs and returning the list of outputs.
model = Model(inputs=(X, s0, c0), outputs=outputs)
    

In [9]:
### List summary of model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X (InputLayer)                  (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17920       X[0][0]                          
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
__________________________________________________________________________________________________
concatenat

activation_9 (Activation)       (None, 30, 1)        0           dense_26[0][0]                   
__________________________________________________________________________________________________
dot_9 (Dot)                     (None, 1, 64)        0           activation_9[0][0]               
                                                                 bidirectional_1[0][0]            
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, 64), (None,  33024       dot_9[0][0]                      
                                                                 lstm_8[0][0]                     
                                                                 lstm_8[0][2]                     
__________________________________________________________________________________________________
repeat_vector_10 (RepeatVector) (None, 30, 64)       0           lstm_9[0][0]                     
__________

In [10]:
### Set up Keras Optimizers
from keras import optimizers

opt = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [11]:
### Set up input parameter as 0
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))

### Prepare train Y data
outputs = list(Yoh.swapaxes(0,1))

In [12]:
### Train compiled model with train data
model.fit([Xoh, s0, c0], outputs, epochs=10, batch_size=100)

Epoch 1/10


 1700/10000 [====>.........................] - ETA: 25:20 - loss: 23.9904 - dense_3_loss: 2.3932 - dense_6_loss: 2.4087 - dense_9_loss: 2.3960 - dense_12_loss: 2.3995 - dense_15_loss: 2.4199 - dense_18_loss: 2.3889 - dense_21_loss: 2.3925 - dense_24_loss: 2.3957 - dense_27_loss: 2.3995 - dense_30_loss: 2.3965 - dense_3_acc: 0.0500 - dense_6_acc: 0.0000e+00 - dense_9_acc: 0.0500 - dense_12_acc: 0.0900 - dense_15_acc: 0.0000e+00 - dense_18_acc: 0.0000e+00 - dense_21_acc: 0.1800 - dense_24_acc: 0.0000e+00 - dense_27_acc: 0.0000e+00 - dense_30_acc: 0.120 - ETA: 12:41 - loss: 23.9435 - dense_3_loss: 2.3887 - dense_6_loss: 2.4022 - dense_9_loss: 2.3960 - dense_12_loss: 2.3970 - dense_15_loss: 2.4083 - dense_18_loss: 2.3826 - dense_21_loss: 2.3930 - dense_24_loss: 2.3829 - dense_27_loss: 2.3947 - dense_30_loss: 2.3980 - dense_3_acc: 0.3550 - dense_6_acc: 0.0350 - dense_9_acc: 0.1200 - dense_12_acc: 0.1050 - dense_15_acc: 0.0000e+00 - dense_18_acc: 0.0150 - dense_21_acc: 0.1550 - dense_24_acc:

 5100/10000 [==============>...............] - ETA: 40s - loss: 18.7516 - dense_3_loss: 1.9363 - dense_6_loss: 1.8539 - dense_9_loss: 2.1174 - dense_12_loss: 2.3972 - dense_15_loss: 1.2443 - dense_18_loss: 1.5047 - dense_21_loss: 2.3523 - dense_24_loss: 1.0992 - dense_27_loss: 1.8878 - dense_30_loss: 2.3585 - dense_3_acc: 0.6177 - dense_6_acc: 0.5977 - dense_9_acc: 0.1886 - dense_12_acc: 0.0931 - dense_15_acc: 0.9166 - dense_18_acc: 0.6983 - dense_21_acc: 0.1309 - dense_24_acc: 0.9711 - dense_27_acc: 0.3074 - dense_30_acc: 0.106 - ETA: 38s - loss: 18.5677 - dense_3_loss: 1.9076 - dense_6_loss: 1.8229 - dense_9_loss: 2.1044 - dense_12_loss: 2.3952 - dense_15_loss: 1.2099 - dense_18_loss: 1.4789 - dense_21_loss: 2.3531 - dense_24_loss: 1.0688 - dense_27_loss: 1.8701 - dense_30_loss: 2.3570 - dense_3_acc: 0.6181 - dense_6_acc: 0.5986 - dense_9_acc: 0.1881 - dense_12_acc: 0.0939 - dense_15_acc: 0.9189 - dense_18_acc: 0.7000 - dense_21_acc: 0.1306 - dense_24_acc: 0.9719 - dense_27_acc: 0.30

 6800/10000 [===================>..........] - ETA: 22s - loss: 16.5011 - dense_3_loss: 1.5554 - dense_6_loss: 1.4769 - dense_9_loss: 1.9597 - dense_12_loss: 2.3719 - dense_15_loss: 0.8382 - dense_18_loss: 1.1992 - dense_21_loss: 2.3312 - dense_24_loss: 0.7403 - dense_27_loss: 1.6832 - dense_30_loss: 2.3451 - dense_3_acc: 0.6117 - dense_6_acc: 0.5983 - dense_9_acc: 0.1854 - dense_12_acc: 0.0952 - dense_15_acc: 0.9438 - dense_18_acc: 0.7177 - dense_21_acc: 0.1390 - dense_24_acc: 0.9806 - dense_27_acc: 0.3173 - dense_30_acc: 0.108 - ETA: 22s - loss: 16.4078 - dense_3_loss: 1.5389 - dense_6_loss: 1.4614 - dense_9_loss: 1.9535 - dense_12_loss: 2.3706 - dense_15_loss: 0.8224 - dense_18_loss: 1.1867 - dense_21_loss: 2.3289 - dense_24_loss: 0.7263 - dense_27_loss: 1.6748 - dense_30_loss: 2.3444 - dense_3_acc: 0.6125 - dense_6_acc: 0.5992 - dense_9_acc: 0.1849 - dense_12_acc: 0.0953 - dense_15_acc: 0.9449 - dense_18_acc: 0.7189 - dense_21_acc: 0.1406 - dense_24_acc: 0.9809 - dense_27_acc: 0.31

 8500/10000 [========================>.....] - ETA: 12s - loss: 15.3058 - dense_3_loss: 1.3378 - dense_6_loss: 1.2775 - dense_9_loss: 1.8751 - dense_12_loss: 2.3568 - dense_15_loss: 0.6319 - dense_18_loss: 1.0366 - dense_21_loss: 2.3179 - dense_24_loss: 0.5580 - dense_27_loss: 1.5773 - dense_30_loss: 2.3369 - dense_3_acc: 0.6157 - dense_6_acc: 0.6055 - dense_9_acc: 0.1935 - dense_12_acc: 0.1001 - dense_15_acc: 0.9577 - dense_18_acc: 0.7317 - dense_21_acc: 0.1430 - dense_24_acc: 0.9854 - dense_27_acc: 0.3241 - dense_30_acc: 0.111 - ETA: 12s - loss: 15.2541 - dense_3_loss: 1.3285 - dense_6_loss: 1.2689 - dense_9_loss: 1.8714 - dense_12_loss: 2.3560 - dense_15_loss: 0.6229 - dense_18_loss: 1.0310 - dense_21_loss: 2.3165 - dense_24_loss: 0.5500 - dense_27_loss: 1.5722 - dense_30_loss: 2.3367 - dense_3_acc: 0.6154 - dense_6_acc: 0.6054 - dense_9_acc: 0.1937 - dense_12_acc: 0.0996 - dense_15_acc: 0.9583 - dense_18_acc: 0.7311 - dense_21_acc: 0.1433 - dense_24_acc: 0.9856 - dense_27_acc: 0.32

...

 1700/10000 [====>.........................] - ETA: 17s - loss: 6.8446 - dense_3_loss: 0.0749 - dense_6_loss: 0.0786 - dense_9_loss: 0.8962 - dense_12_loss: 2.1465 - dense_15_loss: 4.0096e-04 - dense_18_loss: 0.1059 - dense_21_loss: 0.7362 - dense_24_loss: 5.3489e-04 - dense_27_loss: 0.5627 - dense_30_loss: 2.2426 - dense_3_acc: 0.9700 - dense_6_acc: 0.9700 - dense_9_acc: 0.6000 - dense_12_acc: 0.1900 - dense_15_acc: 1.0000 - dense_18_acc: 0.9500 - dense_21_acc: 0.7900 - dense_24_acc: 1.0000 - dense_27_acc: 0.8000 - dense_30_acc: 0.17 - ETA: 18s - loss: 6.8122 - dense_3_loss: 0.0609 - dense_6_loss: 0.0586 - dense_9_loss: 0.8579 - dense_12_loss: 2.1451 - dense_15_loss: 3.9505e-04 - dense_18_loss: 0.0741 - dense_21_loss: 0.7695 - dense_24_loss: 5.3883e-04 - dense_27_loss: 0.6387 - dense_30_loss: 2.2064 - dense_3_acc: 0.9800 - dense_6_acc: 0.9750 - dense_9_acc: 0.5850 - dense_12_acc: 0.2150 - dense_15_acc: 1.0000 - dense_18_acc: 0.9700 - dense_21_acc: 0.7800 - dense_24_acc: 1.0000 - dense

 5100/10000 [==============>...............] - ETA: 12s - loss: 6.6965 - dense_3_loss: 0.0720 - dense_6_loss: 0.0704 - dense_9_loss: 0.8255 - dense_12_loss: 2.1029 - dense_15_loss: 4.0391e-04 - dense_18_loss: 0.0952 - dense_21_loss: 0.6888 - dense_24_loss: 5.0206e-04 - dense_27_loss: 0.6483 - dense_30_loss: 2.1925 - dense_3_acc: 0.9751 - dense_6_acc: 0.9740 - dense_9_acc: 0.6317 - dense_12_acc: 0.2111 - dense_15_acc: 1.0000 - dense_18_acc: 0.9657 - dense_21_acc: 0.7954 - dense_24_acc: 1.0000 - dense_27_acc: 0.7549 - dense_30_acc: 0.19 - ETA: 12s - loss: 6.7024 - dense_3_loss: 0.0730 - dense_6_loss: 0.0713 - dense_9_loss: 0.8263 - dense_12_loss: 2.1036 - dense_15_loss: 4.0365e-04 - dense_18_loss: 0.0958 - dense_21_loss: 0.6903 - dense_24_loss: 5.0590e-04 - dense_27_loss: 0.6485 - dense_30_loss: 2.1926 - dense_3_acc: 0.9747 - dense_6_acc: 0.9736 - dense_9_acc: 0.6300 - dense_12_acc: 0.2108 - dense_15_acc: 1.0000 - dense_18_acc: 0.9653 - dense_21_acc: 0.7933 - dense_24_acc: 1.0000 - dense

 6800/10000 [===================>..........] - ETA: 9s - loss: 6.6935 - dense_3_loss: 0.0741 - dense_6_loss: 0.0723 - dense_9_loss: 0.8257 - dense_12_loss: 2.1016 - dense_15_loss: 4.0299e-04 - dense_18_loss: 0.0933 - dense_21_loss: 0.6881 - dense_24_loss: 5.0714e-04 - dense_27_loss: 0.6480 - dense_30_loss: 2.1896 - dense_3_acc: 0.9729 - dense_6_acc: 0.9727 - dense_9_acc: 0.6319 - dense_12_acc: 0.2125 - dense_15_acc: 1.0000 - dense_18_acc: 0.9658 - dense_21_acc: 0.7942 - dense_24_acc: 1.0000 - dense_27_acc: 0.7590 - dense_30_acc: 0.191 - ETA: 9s - loss: 6.6989 - dense_3_loss: 0.0747 - dense_6_loss: 0.0729 - dense_9_loss: 0.8248 - dense_12_loss: 2.1029 - dense_15_loss: 4.0311e-04 - dense_18_loss: 0.0938 - dense_21_loss: 0.6897 - dense_24_loss: 5.0766e-04 - dense_27_loss: 0.6484 - dense_30_loss: 2.1908 - dense_3_acc: 0.9726 - dense_6_acc: 0.9723 - dense_9_acc: 0.6325 - dense_12_acc: 0.2125 - dense_15_acc: 1.0000 - dense_18_acc: 0.9657 - dense_21_acc: 0.7934 - dense_24_acc: 1.0000 - dense_

 8500/10000 [========================>.....] - ETA: 6s - loss: 6.6711 - dense_3_loss: 0.0766 - dense_6_loss: 0.0750 - dense_9_loss: 0.8187 - dense_12_loss: 2.0973 - dense_15_loss: 3.9937e-04 - dense_18_loss: 0.0920 - dense_21_loss: 0.6785 - dense_24_loss: 5.0706e-04 - dense_27_loss: 0.6448 - dense_30_loss: 2.1872 - dense_3_acc: 0.9713 - dense_6_acc: 0.9714 - dense_9_acc: 0.6329 - dense_12_acc: 0.2133 - dense_15_acc: 1.0000 - dense_18_acc: 0.9665 - dense_21_acc: 0.7933 - dense_24_acc: 1.0000 - dense_27_acc: 0.7636 - dense_30_acc: 0.194 - ETA: 5s - loss: 6.6676 - dense_3_loss: 0.0768 - dense_6_loss: 0.0753 - dense_9_loss: 0.8179 - dense_12_loss: 2.0966 - dense_15_loss: 3.9905e-04 - dense_18_loss: 0.0919 - dense_21_loss: 0.6769 - dense_24_loss: 5.0726e-04 - dense_27_loss: 0.6443 - dense_30_loss: 2.1871 - dense_3_acc: 0.9714 - dense_6_acc: 0.9716 - dense_9_acc: 0.6337 - dense_12_acc: 0.2136 - dense_15_acc: 1.0000 - dense_18_acc: 0.9667 - dense_21_acc: 0.7937 - dense_24_acc: 1.0000 - dense_

10000/10000 [==============================] - ETA: 2s - loss: 6.6510 - dense_3_loss: 0.0766 - dense_6_loss: 0.0750 - dense_9_loss: 0.8134 - dense_12_loss: 2.0988 - dense_15_loss: 3.9479e-04 - dense_18_loss: 0.0899 - dense_21_loss: 0.6646 - dense_24_loss: 5.0063e-04 - dense_27_loss: 0.6476 - dense_30_loss: 2.1842 - dense_3_acc: 0.9716 - dense_6_acc: 0.9716 - dense_9_acc: 0.6377 - dense_12_acc: 0.2147 - dense_15_acc: 1.0000 - dense_18_acc: 0.9671 - dense_21_acc: 0.7963 - dense_24_acc: 1.0000 - dense_27_acc: 0.7644 - dense_30_acc: 0.196 - ETA: 2s - loss: 6.6519 - dense_3_loss: 0.0766 - dense_6_loss: 0.0750 - dense_9_loss: 0.8144 - dense_12_loss: 2.0985 - dense_15_loss: 3.9455e-04 - dense_18_loss: 0.0900 - dense_21_loss: 0.6651 - dense_24_loss: 5.0238e-04 - dense_27_loss: 0.6472 - dense_30_loss: 2.1843 - dense_3_acc: 0.9717 - dense_6_acc: 0.9717 - dense_9_acc: 0.6372 - dense_12_acc: 0.2147 - dense_15_acc: 1.0000 - dense_18_acc: 0.9669 - dense_21_acc: 0.7961 - dense_24_acc: 1.0000 - dense_

In [13]:
## Import already trained model
#model.load_weights('models/model.h5')

In [60]:
### Test with some example data
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    #source1 = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source1 = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source2 = np.reshape(source1, (1, 30, 37))
    prediction = model.predict([source2, s0, c0])
    prediction2 = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction2]
    
    print("source:", example)
    print("output:", ''.join(output))

source: 3 May 1979
output: 1999-05-09
source: 5 April 09
output: 1970-04-04
source: 21th of August 2016
output: 2013-03-03
source: Tue 10 Jul 2007
output: 2000-08-10
source: Saturday May 9 2018
output: 1912-05-07
source: March 3 2001
output: 2003-03-03
source: March 3rd 2001
output: 2000-03-03
source: 1 March 2001
output: 2003-03-11
